# Hits, misses and false alarms
***

**Author**: Chus Casado Rodríguez<br>
**Date**: 14-02-2024<br>


**Introduction**:<br>
This notebooks computes the hits (true positives, $TP$), misses (false negatives, $FN$) and false alarms (false positives, $FP$) for all the selected reporting points and the complete study period.

The input data are the data sets of exceedance over threshold both for the reanalsysis ("observed") and the forecast.

The confusion matrix (hits, misses, false alarms) is computed for all the possible combinations of three criteria: model combination method (named _approach_), persistence, and probability threshold. The values of persisntece and probability thresholds for which the confusion matrix will be evaluated are defined by the user.

The result is a new set of NetCDF files (one for station) that contains matrixes of hits, misses and false alarms for every combination of the criteria.

**Interesting links**<br>
[Evaluation metrics for imbalanced classification](https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/)<br>
[Cross entropy for machine learning](https://machinelearningmastery.com/cross-entropy-for-machine-learning/)<br>
[Probability metrics for imbalanced classification](https://machinelearningmastery.com/probability-metrics-for-imbalanced-classification/)<br>
[ROC curves and precision-recall curves for imbalanced classification](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/)<br>
[Instructions for sending EFAS flood notifications](https://efascom.smhi.se/confluence/display/EDC/Instructions+for+sending%2C+upgrading+and+deactivating+EFAS+Flood+Notifications)

In [1]:
import os
path_root = os.getcwd()
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import ListedColormap
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import yaml
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

os.chdir('../py/')
from config import Config
from compute import *
from seasonal import disaggregate_by_season
from convert import reshape_DataArray, dict2da
from plot.results import plot_DataArray, plot_weights, plot_brier_skill
from plot.maps import create_cmap, map_events
os.chdir(path_root)

In [2]:
# Set the default text font size
plt.rc('font', size=15)
# Set the axes title font size
plt.rc('axes', titlesize=16)
# Set the axes labels font size
plt.rc('axes', labelsize=15)
# Set the font size for x tick labels
plt.rc('xtick', labelsize=13)
# Set the font size for y tick labels
plt.rc('ytick', labelsize=13)
# Set the legend font size
plt.rc('legend', fontsize=13)
# Set the font size of the figure title
plt.rc('figure', titlesize=17)

## 1 Configuration

In [3]:
config_path = Path('../conf')
config = Config.load_from_yaml(config_path / 'config_COMB_all_leadtimes.yml')

### 1.1 Reporting points

In [4]:
# area threshold
area_threshold = config.reporting_points['area']

# fixed notification criteria for the optimization process
area_optimization = config.skill['area']

# reporting points
path_stations = config.reporting_points['output']
file_stations = path_stations / f'reporting_points_over_{area_threshold}km2.parquet'

### 1.2 Exceedance

In [6]:
# return period
rp = config.discharge['return_period']['threshold']

# path where results from the preprocessing steps were saved
path_reanalysis = config.exceedance['output']['reanalysis'] / f'{rp}'
path_forecast = config.exceedance['output']['forecast'] / f'{rp}'

### 1.1 Hits

In [7]:
# probability thresholds
probability_range = config.hits['criteria']['probability']
probability = np.arange(*probability_range).round(3)
probability = xr.DataArray(probability, dims=['probability'], coords={'probability': probability})

# persistence
persistence = config.hits['criteria']['persistence']
persistence = {'/'.join([str(i) for i in pers]): pers for pers in persistence}

# lead time ranges
leadtime = config.hits['leadtime']

# parameters of the rolling window used to compute hits
window = config.hits['window']
center = config.hits['center']

# dissagregate the analysis by seasons?
seasonality = config.hits['seasonality']

# path where the dataset of hits, misses and false alarms will be saved
path_out_root = config.hits['output']
path_out_root = Path(path_out_root) / f'{rp}'
path_weights = path_out_root / 'COMB' / 'weighing'
if leadtime is None:
    folder = 'all_leadtimes'
elif len(leadtime) == 10:
    folder = 'daily'
elif len(leadtime) == 20:
    folder = '12h'
else:
    folder = '_'.join([str(lt + 12) for lt in leadtime])
path_out = path_out_root / 'COMB' / f'{folder}'
if seasonality:
    path_out += 'seasonal/'
for path in [path_out_root, path_out, path_weights]:
    path.mkdir(parents=True, exist_ok=True)

## 2 Data

### 2.1 Reporting points

I load all the stations that where selected in a previous [notebook](3_0_select_stations.ipynb).

In [ ]:
# load table of fixed reporing points
stations = pd.read_parquet(file_stations)

### 2.2 Exceedance reanalysis

In [ ]:
# load probability of exceeding the discharge threshold in the REANALYSIS data
rean_exc = xr.open_mfdataset(f'{path_reanalysis}/*.nc', combine='nested', concat_dim='id')['exceedance']
rean_exc = rean_exc.sel(id=stations.index).compute()

if seasonality:
    rean_exc = disaggregate_by_season(rean_exc)

print(rean_exc.dims)
print(rean_exc.shape)

### 2.3 Exceedance forecast

In [ ]:
# load probability of exceeding the discharge threshold in the FORECAST data
fore_exc = xr.open_mfdataset(f'{path_forecast}/*.nc', combine='nested', concat_dim='id')
fore_exc['id'] = fore_exc.id.astype(int)
fore_exc = fore_exc.sel(id=stations.index)

# reshape the DataArray of forecasted exceedance
fore_exc = xr.Dataset({label: reshape_DataArray(da, trim=True) for label, da in fore_exc.items()})
fore_exc = fore_exc.transpose('id', 'datetime', 'leadtime', 'model')

# extract starting and ending dates
if ('start' not in locals()) or ('end' not in locals()):
    start = pd.to_datetime(max(rean_exc.datetime.min().data, fore_exc.datetime.min().data))
    end = pd.to_datetime(min(rean_exc.datetime.max().data, fore_exc.datetime.max().data))
else:
    start = max(start, rean_exc.datetime.min(), fore_exc.datetime.min())
    end = min(end, rean_exc.datetime.max(), fore_exc.datetime.max())

# recalculate the exceedance datasets to convert the 3 classes (>Q5, >0.95·Q5, <0.95·Q5) to only 2 (exceedance, non-exceedance)
rean_exc, fore_exc = recompute_exceedance(rean_exc.sel(datetime=slice(start, end)),
                                          fore_exc['high'].sel(datetime=slice(start, end)),
                                          fore_exc['low'].sel(datetime=slice(start, end)))

### 2.4 Weighting factors

**Brier weighting**

In [ ]:
# COMPUTE BRIER SCORE
brier = {}
for model in fore_exc.model.data:
    # extract model forecast
    pred =  fore_exc.sel(model=model).dropna('leadtime', how='any').transpose()
    # squared error
    se = (pred - rean_exc)**2
    # Brier score
    brier[model] = se.mean(['id', 'datetime'])

# convert dictionary to DatArray
brier = dict2da(brier, dim='model')

# CONVERT BRIER SCORE INTO WEIGHTING FACTORS
# e = 1e-5
# weights_brier = (brier.max() + e - brier) / (brier.max() + e - (brier.min() - e))
w = 7
weights_brier = brier**-w
weights_brier /= weights_brier.sum('model')

# export
brier.to_netcdf(path_weights / 'brier_scores.nc')
weights_brier.to_netcdf(path_weights / 'weights_brier.nc')

In [ ]:
# climatology
Pclim = 0
se= (Pclim - rean_exc)**2
brier_clim = se.mean(['id', 'datetime'])

# Brier skill score
BSS = (brier_clim - brier) / brier_clim

In [ ]:
# create a copy of 'skill_opt' to change acronyms
ds = BSS.copy()
ds['model'] = ds['model'].where(ds['model'] != 'EUD', 'HRES')
ds['model'] = ds['model'].where(ds['model'] != 'EUE', 'ENS')
ds = ds.reindex(model=['ENS', 'COS', 'HRES', 'DWD'])

# plot the Brier skill score
top = cm.get_cmap('Blues_r', 128)
bottom = cm.get_cmap('Oranges', 128)
newcolors = np.vstack((top(np.linspace(0.2, .95, 128)),
                       bottom(np.linspace(.05, .8, 128))))
BuOr = ListedColormap(newcolors, name='BlueOrange')

df = plot_brier_skill(ds,
                      cmap=BuOr,
                      ylim=(-.12, 1.02),
                      save=path_weights / 'Brier_skill_score.pdf')

>***Figure 1**. Brier skill score for every meteorological model and lead time. The benchmark is a model whose probability of exceedance is 0.*

**Member weighting**

In [ ]:
# compute weights by number of members
weights_member = weights_brier.copy()
for model in models:
    weights_member.loc[{'model': model}] = weights_member.sel(model=model).where(weights_brier.sel(model=model).isnull(), models[model]['members'])
weights_member /= weights_member.sum('model')

# export
weights_member.to_netcdf(path_weights / 'weights_member.nc')

**Model mean**

In [ ]:
# compute weights by number of members
weights_mean = weights_brier.copy()
weights_mean = weights_mean.where(weights_mean.isnull(), other=1)
weights_mean /= weights_mean.sum('model')

# export
weights_mean.to_netcdf(path_weights / 'weights_mean.nc')

**Compare weighing methods**

In [ ]:
# heatmap of weights
Weights = xr.Dataset({'model mean': weights_mean, 'member weighted': weights_member, 'Brier weighted': weights_brier})
Weights['model'] = Weights['model'].where(Weights['model'] != 'EUD', 'HRES')
Weights['model'] = Weights['model'].where(Weights['model'] != 'EUE', 'ENS')
Weights = Weights.reindex(model=['ENS', 'COS', 'HRES', 'DWD'])

plot_weights(Weights,
             cmap=BuOr,
             offset=-12,
             save=path_weights / 'weights.pdf')

>***Figure 2**. Weights assigned to each NWP model and lead time in the three total probability approaches.*

## 3 Analysis

### 3.1 Observed events
#### 3.1.1 Number of events

In [ ]:
# find the onset of the events
onsets =  xr.concat((rean_exc.isel(datetime=0).astype(bool), rean_exc.diff('datetime')), dim='datetime') == 1
onsets = onsets.sel(datetime=slice(start, end))

# find the offset of the events
offsets = xr.concat((rean_exc.isel(datetime=0).astype(bool), rean_exc.diff('datetime')), dim='datetime') == -1
offsets = offsets.sel(datetime=slice(start, end))

# count number of events
col_events = f'obs_events_{rp}'
stations[col_events] = onsets.sum('datetime').to_pandas()

print('No. stations with observed events:\t{0}'.format((stations[col_events] > 0).sum()))
print('No. observed events:\t\t\t{0}'.format(stations[col_events].sum()))

if seasonality:
    # compute number of events per season
    rean_onsets4s = disaggregate_by_season(rean_onsets, dim='datetime')
    cols = [f'obs_events_{rp}_{season}' for season in ['winter', 'spring', 'summer', 'autumn']]
    stations[cols] = rean_onsets4s.sum('datetime').to_pandas().transpose()

# export the stations table
stations.to_parquet(file_stations)

In [ ]:
# map of the number of events
map_events(stations.X,
           stations.Y, 
           stations[col_events],
           yscale='log', alpha=1,
           save=path_out_root / f'map_observed_events_{stations.shape[0]}points.jpg')

> ***Figure 3**. Number of observed flood events during the study period.*

The geographical distribution of events is not even. There is a higher proportion of stations with events in Central Europe, British Isles and the Mediterranean catchments than in Estearn and North-Eastern Europe. During the study period there were major events in the Rhine, Meuse and Ebro, which can be seen in the map.

The reporting points with more than 5 flood events during the study period were removed, since it is suspicious that the 5-year return period was exceeded so many times in only 2 years of study period.

#### 3.1.2 Duration of the events

In [ ]:
# evaluate the duration (in hours) of each event
event_durations = {}
for stn in tqdm(onsets.id.data):
    if stations.loc[stn, col_events] == 0:
        continue
    # dates of the onsets and offsets of the events
    dates_onsets = onsets.sel(id=stn).where(onsets.sel(id=stn), drop=True).datetime.data
    dates_offsets = offsets.sel(id=stn).where(offsets.sel(id=stn), drop=True).datetime.data
    # corrections in case the amount of onsets and onsets don't match
    if len(dates_onsets) > len(dates_offsets):
        dates_offsets = np.append(dates_offsets, offsets.datetime[-1].data)
    elif len(dates_onsets) < len(dates_offsets):
        dates_onsets = np.append(onsets.datetime[0].data, dates_onsets)
    # correction in case the first onset is later that the first offset
    else:
        if dates_onsets[0] > dates_offsets[0]:
            print(stn)
            dates_onsets = np.append(onsets.datetime[0].data, dates_onsets)
            dates_offsets = np.append(dates_offsets, offsets.datetime[-1].data)
    # convert into hours
    durations = (dates_offsets - dates_onsets) / np.timedelta64(1, 'h')
    event_durations[stn] = durations

In [ ]:
# convert dictionary of durations into a flattened array
durations_all, durations_opt = [], []
for stn, arr in event_durations.items():
    for d in arr:
        durations_all.append(d)
        if stations.loc[stn, 'area'] >= area_optimization:
            durations_opt.append(d)
durations_all = np.array(durations_all)
durations_opt = np.array(durations_opt)

In [ ]:
# plot distribution of event duration
alpha = .7
xmin = 0
xmax = 16 * 24
bins = np.arange(xmin, xmax + 1, 6).astype(int)

fig, ax = plt.subplots(figsize=(5, 5))
sns.histplot(durations_all, ax=ax, alpha=alpha**2, bins=bins, color='steelblue', edgecolor=None, label=f'A ≥ {area_threshold} km²')
sns.histplot(durations_opt, ax=ax, alpha=alpha, bins=bins, color='orange', edgecolor=None, label=f'A ≥ {area_optimization} km²')
ax.set(xlabel='duration (h)', ylabel='no. events', xlim=(xmin, xmax));
ax.spines[['right', 'top']].set_visible(False)
ax.xaxis.set_major_locator(MultipleLocator(96))
ax.xaxis.set_minor_locator(MultipleLocator(24))
ax.yaxis.set_major_locator(MultipleLocator(100))
ax.yaxis.set_minor_locator(MultipleLocator(25))
ax.legend(frameon=False);

plt.savefig(path_out_root / f'duration_distribution_{stations.shape[0]}points.jpg',
            dpi=300, bbox_inches='tight')

>***Figure 4**. Distributions of the duration of the observed flood events. In blue the events observed in points with a catchment area of at least 500 km², in orage those that exceed 2000 km².*

### 3.2 Hits, misses and false alarms

In [ ]:
path_netcdf = path_out / f'window_{window}'
path_netcdf.mkdir(parents=True, exist_ok=True)

for stn in tqdm(stations.index):

    # check if the output file already exists
    file_out = path_netcdf / f'{stn:>04}.nc'
    if os.path.exists(file_out):
        continue
        
    # FORECAST EXCEEDANCE PROBABILITY
    forecast = fore_exc.sel(id=stn)

    # TOTAL PROBABILITY OF EXCEEDANCE

    # exceedance according to 1 deterministic + 1 probabilistic
    deterministic = (forecast.sel(model=['EUD', 'DWD']) >= probability).any('model')
    probabilistic = (forecast.sel(model=['EUE', 'COS']) >= probability).any('model')
    deterministic_probabilistic = deterministic & probabilistic

    # exceedance according to mean over models
    model_mean = forecast.mean('model', skipna=True) >= probability

    # exceedance according to the mean over models weighted by the number of members
    member_weighted = forecast.weighted(weights_member.fillna(0)).mean('model', skipna=True) >= probability

    # exceedance according to the mean over models weighted by the inverse Brier score
    brier_weighted = forecast.weighted(weights_brier.fillna(0)).mean('model', skipna=True) >= probability

    # merge all total probability approaches in a single DataArray
    total_exc = xr.Dataset({
                            '1_deterministic_+_1_probabilistic': deterministic_probabilistic,
                            'model_mean': model_mean,
                            'member_weighted': member_weighted,
                            'brier_weighted': brier_weighted,
                            }).to_array(dim='approach')

    del forecast

    # HITS, MISSES, FALSE ALARMS
    
    hits = {}
    for label, pers in persistence.items():

        # compute predicted events
        pred = exceedance2events(total_exc, persistence=pers, leadtime=leadtime)

        # disaggregate seasonaly
        if seasonality:
            pred = disaggregate_by_season(pred)

        # compute hits, misses and false alarms
        # if 'leadtime' in pred.dims:
        #     aux = events2hits(rean_exc.sel(id=stn), pred, center=center, w=window)
        # else:
        #     aux = events2hits(rean_exc, pred, center=center, w=window)
        hits[label] = events2hits(rean_exc.sel(id=stn), pred, center=center, w=window)
    hits = dict2da(hits, dim='persistence')

    print(f'Exporting file {file_out}', end='\r')
    hits.to_netcdf(file_out)

    del pred, hits